In [27]:
# Carga de biliotecas 

import os
import pyodbc
import pandas as pd
from   dotenv import load_dotenv
from   datetime import datetime
from   sqlalchemy import create_engine
from   sqlalchemy import text


In [4]:
# conexao com banco de dados origem

def connectarDBtransacional():

    load_dotenv()

    usuario  = os.getenv('DB_USER')
    servidor = os.getenv('DB_SERVER')
    senha    = os.getenv('DB_PASSWORD')
    banco    = os.getenv('DB_NAME')
    driver   = os.getenv('DB_DRIVER')

    stringConexao =  f'mssql+pyodbc://{usuario}:{senha}@{servidor}/{banco}?driver={driver}'

    engine = create_engine(stringConexao)

    return engine

In [29]:
# Extracao de dados ultimos 12 meses de vendas, janeiro a novembroDezembro 2015 toda a nossa base )

query_vendas_passados = """
    SELECT 
        V.DOCUMENTO AS FATURA,
        V.FECHA AS DATA,
        V.TIPO_DOCUMENTO AS 'TIPO DOCUMENTO',
        CONCAT(C.NOMBRE, ' ', C.APELLIDO) AS CLIENTE,
        CONCAT(VE.NOMBRE,'',VE.APELLIDO) AS VENDEDOR,
        C.ESTADO_CIVIL AS 'ESTADO CIVIL CLIENTE',
        C.EDAD AS IDADE_CLIENTE,
        A.NOMBRE AS PRODUTO,
        A.FAMILIA,
        D.CANTIDAD AS QUANTIDADE,
        D.PRECIO_UNITARIO AS PRECO_UNITARIO,
        V.IMPUESTO AS IMPOSTO,
        S.NOMBRE_SUCURSAL AS FILIAL,
        CO.COMUNA_NOMBRE AS COMUNIDADE_FILIAL,
        P.PROVINCIA_NOMBRE AS PROVINCIAL_FILIAL,
        R.REGION AS REGION_FILIAL,
        DATENAME(month, FECHA) AS NOME_MES,
        DATEPART(DAY,   FECHA) AS DIA,
        DATENAME(WEEKDAY, FECHA) AS DIA_SEMANA,
        DATEPART(MONTH, FECHA) AS MES_NUMERO,
        DATEPART(YEAR,  FECHA) AS ANO
    FROM 
        VENTAS AS V 
        LEFT JOIN DETALLE_VENTA AS D ON V.DOCUMENTO = D.DOCUMENTO
        LEFT JOIN VENDEDORES    AS VE ON VE.ID_VENDEDOR = V.ID_VENDEDOR
        LEFT JOIN CLIENTES      AS C ON C.ID_CLIENTE = V.ID_CLIENTE
        LEFT JOIN SUCURSAL      AS S ON S.SUCURSAL_ID = V.SUCURSAL_ID
        LEFT JOIN ARTICULOS     AS A ON A.CODIGO_PRODUCTO = D.CODIGO_PRODUCTO
        LEFT JOIN COMUNA        AS CO ON CO.COMUNA_ID = S.COMUNA_ID
        LEFT JOIN PROVINCIA     AS P ON P.PROVINCIA_ID = CO.PROVINCIA_ID
        LEFT JOIN REGION        AS R ON R.REGION_ID = P.REGION_ID
    WHERE
        V.FECHA <= '2015-12-31';
"""

# Extracao de dados mes atual e ano atual da atividade;

hoje = datetime.now()
ano = hoje.year
mes = hoje.month
nome_mes = hoje.strftime("%B")

query_vendas_atual = f"""
    SELECT 
        V.DOCUMENTO AS FATURA,
        V.FECHA AS DATA,
        V.TIPO_DOCUMENTO AS 'TIPO DOCUMENTO',
        CONCAT(C.NOMBRE, ' ', C.APELLIDO) AS CLIENTE,
        CONCAT(VE.NOMBRE,'',VE.APELLIDO) AS VENDEDOR,
        C.ESTADO_CIVIL AS 'ESTADO CIVIL CLIENTE',
        C.EDAD AS IDADE_CLIENTE,
        A.NOMBRE AS PRODUTO,
        A.FAMILIA,
        D.CANTIDAD AS QUANTIDADE,
        D.PRECIO_UNITARIO AS PRECO_UNITARIO,
        V.IMPUESTO AS IMPOSTO,
        S.NOMBRE_SUCURSAL AS FILIAL,
        CO.COMUNA_NOMBRE AS COMUNIDADE_FILIAL,
        P.PROVINCIA_NOMBRE AS PROVINCIAL_FILIAL,
        R.REGION AS REGION_FILIAL,
        DATENAME(month, FECHA) AS NOME_MES,
        DATEPART(DAY,   FECHA) AS DIA,
        DATENAME(WEEKDAY, FECHA) AS DIA_SEMANA,
        DATEPART(MONTH, FECHA) AS MES_NUMERO,
        DATEPART(YEAR,  FECHA) AS ANO
    FROM 
        VENTAS AS V 
        LEFT JOIN DETALLE_VENTA AS D ON V.DOCUMENTO = D.DOCUMENTO
        LEFT JOIN VENDEDORES    AS VE ON VE.ID_VENDEDOR = V.ID_VENDEDOR
        LEFT JOIN CLIENTES      AS C ON C.ID_CLIENTE = V.ID_CLIENTE
        LEFT JOIN SUCURSAL      AS S ON S.SUCURSAL_ID = V.SUCURSAL_ID
        LEFT JOIN ARTICULOS     AS A ON A.CODIGO_PRODUCTO = D.CODIGO_PRODUCTO
        LEFT JOIN COMUNA        AS CO ON CO.COMUNA_ID = S.COMUNA_ID
        LEFT JOIN PROVINCIA     AS P ON P.PROVINCIA_ID = CO.PROVINCIA_ID
        LEFT JOIN REGION        AS R ON R.REGION_ID = P.REGION_ID
    WHERE
        YEAR (V.FECHA) = {ano} AND MONTH (V.FECHA) = {mes};
"""

engine = connectarDBtransacional()

if not os.path.exists('..\\dados_extraidos\\vendas_passados.parquet'):
    
    df_vendas_passados = pd.read_sql(query_vendas_passados,  con=engine)
    df_vendas_passados.to_parquet('..\\dados_extraidos\\vendas_passados.parquet', index = False)

else:
    
    df_vendas_atual = pd.read_sql(query_vendas_atual,  con=engine)

    # verifica se existe novos dados antes de exportar
    if not df_vendas_atual.empty:
        df_vendas_atual.to_parquet(f'..\\dados_extraidos\\vendas_atual-{nome_mes}-{ano}.parquet', index = False)


# criar um funcao envio mensagem sucesso e arquivo log de extracao de dados
